#### Observe that we can get news of a day with the link format:
http://news.ltn.com.tw/list/newspaper/politics/20190101
#### We need a list of date string in the date range we want.

In [1]:
from datetime import datetime, timedelta

start_date = "2019-01-01"
stop_date = "2019-02-28"

start = datetime.strptime(start_date, "%Y-%m-%d")
stop = datetime.strptime(stop_date, "%Y-%m-%d")

dates = list()
while start <= stop:
    dates.append(start.strftime('%Y%m%d'))
    start = start + timedelta(days=1)


#### Now write a function to parse the HTML response, return the data we want(title, content, ...etc).

In [2]:
import requests
from bs4 import BeautifulSoup as bs

In [3]:
def process_document(document, date):
    
    nodes = document.select('ul.list > li')
    data = list()

    for li in nodes:

        # check if is empty element
        if li.select_one('a') == None:
            continue

        # get link
        li_link = 'http://news.ltn.com.tw/' + li.select_one('a')['href']

        # request for document
        li_res = requests.get(li_link)
        li_doc = bs(li_res.text, 'lxml')

        # get date
        li_date = datetime.strptime(date, "%Y%m%d").strftime('%Y-%m-%d')

        #get title
        li_title = li.select_one('p').get_text()

        #get content
        li_content = ""
        for ele in li_doc.select('div.text > p'):
            if not 'appE1121' in ele.get('class', []):
                li_content += ele.get_text()
        
        #get keywords
        li_keyword = li_doc.find(attrs={"name":"keywords"})['content']

        # append new row
        data.append({
            'date' : li_date,
            'title': li_title,
            'link' : li_link,
            'content' : li_content,
            'keywords' : li_keyword,
            'tags' : []
        })
    return data

#### Crawl over the news on the site, store the data in variable "all_data" .

In [ ]:
cnt = 0
all_data = list()
for date in dates:
    print('start crawling :', date)
    res = requests.get('https://news.ltn.com.tw/list/newspaper/politics/' + date)
    doc = bs(res.text, 'lxml')
    data = process_document(doc, date)
    all_data += data

start crawling : 20190101
start crawling : 20190102
start crawling : 20190103
start crawling : 20190104
start crawling : 20190105
start crawling : 20190106
start crawling : 20190107
start crawling : 20190108
start crawling : 20190109
start crawling : 20190110
start crawling : 20190111
start crawling : 20190112
start crawling : 20190113
start crawling : 20190114
start crawling : 20190115
start crawling : 20190116
start crawling : 20190117
start crawling : 20190118
start crawling : 20190119
start crawling : 20190120
start crawling : 20190121
start crawling : 20190122
start crawling : 20190123
start crawling : 20190124
start crawling : 20190125
start crawling : 20190126
start crawling : 20190127
start crawling : 20190128
start crawling : 20190129
start crawling : 20190130
start crawling : 20190131
start crawling : 20190201
start crawling : 20190202
start crawling : 20190203
start crawling : 20190204
start crawling : 20190205
start crawling : 20190206
start crawling : 20190207
start crawli

#### Check the result

In [ ]:
all_data[0:5]

#### Save as pkl file

In [ ]:
import pickle

with open('data/liberty_times_copy_2.pkl', 'wb') as f:
    pickle.dump(all_data, f)

#### Turn it into pandas dataframe

In [ ]:
import pandas as pd
pd.DataFrame(all_data)[['date', 'title', 'link', 'content','keywords', 'tags']].head()